## Start

In [11]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[total_active_users_first_mention]]


### Table

In [15]:
query_text = """--sql
CREATE TABLE db1.total_active_users_rep_mobile_total
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `total_active_users` UInt32,
    `new_active_users` UInt32,
    `total_active_users_monetization` UInt32,
    `total_active_user_subscribed_monetization` UInt32,
    `total_active_users_ble_available` UInt32,
    `total_active_users_ble_available_monetization` UInt32,
    `total_active_users_ble_available_subscribed_monetization` UInt32
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [13]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 6 HOUR 8 MINUTE TO db1.total_active_users_rep_mobile_total AS
    WITH dec_t AS (SELECT
            DISTINCT
            t1.report_date  AS report_date,
            t2.city  AS city,
            t2.partner_uuid  AS partner_uuid
        FROM
            (SELECT DISTINCT report_date FROM db1.total_active_users_first_mention) AS t1
        CROSS JOIN
            (SELECT DISTINCT city, partner_uuid FROM db1.total_active_users_first_mention ) AS t2
        ),
        --
        dec_and_full AS (
        SELECT
        dec_t.report_date AS report_date,
        dec_t.partner_uuid AS partner_uuid,
        dec_t.city AS city,
        total_active_users_day,
        new_active_users_day,
        total_active_users_monetization_day,
        total_active_user_subscribed_monetization_day,
        total_active_users_ble_available_day,
        total_active_users_ble_available_monetization_day,
        total_active_users_ble_available_subscribed_monetization_day
        FROM dec_t
        LEFT join (
            SELECT
                report_date,
                partner_uuid,
                city,
                count(DISTINCT citizen_id) AS total_active_users_day,
                count(DISTINCT if(toDateOrNull(activated_at) BETWEEN toStartOfMonth(report_date) and report_date,citizen_id,null)) AS new_active_users_day,
                count(DISTINCT if(monetization = 1,citizen_id,null)) AS total_active_users_monetization_day,
                count(DISTINCT if(monetization = 1 and subscriptions_state = 'activated',citizen_id,null)) as total_active_user_subscribed_monetization_day,
                count(DISTINCT if(ble_available = 'true',citizen_id,null)) AS total_active_users_ble_available_day,
                count(DISTINCT if(ble_available = 'true' and monetization = 1,citizen_id,null)) AS total_active_users_ble_available_monetization_day,
                count(DISTINCT if(ble_available = 'true' and monetization = 1 and subscriptions_state = 'activated',citizen_id,null)) AS total_active_users_ble_available_subscribed_monetization_day
            FROM db1.total_active_users_first_mention
            GROUP BY
                report_date,
                partner_uuid,
                city) AS full_table
            ON dec_t.report_date = full_table.report_date
            AND dec_t.partner_uuid = full_table.partner_uuid
            AND dec_t.city = full_table.city
        )
        --
        SELECT
            report_date,
            partner_uuid,
            city,
            sum(total_active_users_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS total_active_users,
            sum(new_active_users_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS new_active_users,
            sum(total_active_users_monetization_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS total_active_users_monetization,
            sum(total_active_user_subscribed_monetization_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS total_active_user_subscribed_monetization,
            sum(total_active_users_ble_available_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date) AS total_active_users_ble_available,
            sum(total_active_users_ble_available_monetization_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date)
                AS total_active_users_ble_available_monetization,
            sum(total_active_users_ble_available_subscribed_monetization_day) OVER (PARTITION BY toStartOfMonth(report_date),partner_uuid, city ORDER BY report_date)
                AS total_active_users_ble_available_subscribed_monetization
        FROM dec_and_full
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [17]:
query_text = """--sql
    SELECT
        report_date,
        sum(new_active_users)
    FROM db1.total_active_users_rep_mobile_total
    GROUP BY report_date
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,sum(new_active_users)
date,i64
2025-08-14,13433
2025-08-13,12347
2025-08-12,11122
2025-08-10,9491
2025-08-09,8632
…,…
2025-05-10,8412
2025-05-09,7707
2025-05-08,7007


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.total_active_users_per_day_full_table DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

### drop mv

In [12]:
query_text = """--sql
    DROP TABLE db1.total_active_users_rep_mobile_total_mv
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Table db1.total_active_users_rep_mobile_total_mv does not exist. Maybe you meant db1.total_active_users_rep_mobile_total_mv?. (UNKNOWN_TABLE) (version 25.3.3.42 (official build))


### drop ch

In [14]:
query_text = """--sql
    DROP TABLE db1.total_active_users_rep_mobile_total
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_active_users_per_day_full_table_mv
"""

ch.query_run(query_text)